In [1]:
import os

In [2]:
%pwd

'd:\\Satyam Projects\\Chicken Disease\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Satyam Projects\\Chicken Disease\\Chicken-Disease-Classification'

Below is the code for the creation of the entity. It will differ as the per requirement of the project.

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)        #Its an entity as it will help to create your own custom return type
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

Below is the part of "Update the configuration manager in the src config"

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,       #Will get this value from constants folder you have
        params_filepath = PARAMS_FILE_PATH):      #Will get this value from constants folder you have

        self.config = read_yaml(config_filepath)      #Calling the read_yaml method which we have defined in the common.py in utils folder
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])  #artifacts roots is the key defined in the config.yaml


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: #Return type will be an entity defined above
        config = self.config.data_ingestion   #This will store data ingestion key which 4 other key in it. You can see the same in config.yaml
        #As the return type of the read_yaml is config box so we can access key using "."

        create_directories([config.root_dir])   

        data_ingestion_config = DataIngestionConfig(         #Using the entity you have created above
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import gdown

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            gdown.download(self.config.source_URL, self.config.local_data_file, quiet=False, fuzzy=True)
            #logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        #zipfile.ZipFile(self.config.local_data_file, 'r').testzip()
        with zipfile.ZipFile(self.config.local_data_file) as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
#os.chdir("D:\Satyam Projects\Chicken Disease\Chicken-Disease-Classification")
%pwd

'd:\\Satyam Projects\\Chicken Disease\\Chicken-Disease-Classification'

In [11]:
import time

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-08 09:46:45,191: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-08 09:46:45,195: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-08 09:46:45,196: INFO: common: created directory at: artifacts]
[2023-07-08 09:46:45,200: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From: https://drive.google.com/uc?id=1Toa-TOiSYJ-fAzU_TYnSgRAhAGfqix_F
To: d:\Satyam Projects\Chicken Disease\Chicken-Disease-Classification\artifacts\data_ingestion\TrainingImages.zip
100%|██████████| 24.9M/24.9M [00:05<00:00, 4.73MB/s]
